In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, BatchSampler, RandomSampler

torch.manual_seed(1)

swap_data = pd.read_csv('swap_pair_v2.csv')
pair_list = list(map(tuple,swap_data.values))#[:100000]
styles = set(list(swap_data['ORIGIN_STYLE']) + list(swap_data['REPLACE_STYLE']))
style_to_idx = {style: i for i, style in enumerate(styles)}
print(len(pair_list), len(style_to_idx))

EMBEDDING_DIM = 10

238191 16420


In [ ]:
# class StyleEmbedModeler(torch.nn.Module):
    
#     def __init__(self, style_size, embedding_dim):
#         super(StyleEmbedModeler, self).__init__()
#         self.embeddings = torch.nn.Embedding(style_size, embedding_dim)
#         self.linear1 = torch.nn.Linear(embedding_dim, 128)
#         self.linear2 = torch.nn.Linear(128, style_size)

#     def forward(self, inputs):
#         embeds = self.embeddings(inputs).view((1, -1))
#         out = torch.nn.functional.relu(self.linear1(embeds))
#         out = self.linear2(out)
#         log_probs = torch.nn.functional.log_softmax(out, dim=1)
#         return log_probs

# losses = []
# loss_function = torch.nn.NLLLoss()
# model = StyleEmbedModeler(len(styles), EMBEDDING_DIM)
# optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5, weight_decay = 0.0005)
# BATCH_SIZE = 512
# data_loader = DataLoader(pair_list, batch_size = BATCH_SIZE, shuffle = True)

# for epoch in range(10):
#     data_loader = DataLoader(pair_list, batch_size = BATCH_SIZE, shuffle = True)
#     for item1s, item2s in data_loader:
#         print('----------------------------------')
#         total_loss = 0
#         for i in range(BATCH_SIZE):
#             item1 = item1s[i]
#             item2 = item2s[i]
#             # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
#             # into integer indices and wrap them in tensors)
#             context_idxs = torch.tensor([style_to_idx[item1]], dtype=torch.long)

#             # Step 2. Recall that torch *accumulates* gradients. Before passing in a
#             # new instance, you need to zero out the gradients from the old
#             # instance
#             model.zero_grad()

#             # Step 3. Run the forward pass, getting log probabilities over next
#             # words
#             log_probs = model(context_idxs)

#             # Step 4. Compute your loss function. (Again, Torch wants the target
#             # word wrapped in a tensor)
#             loss = loss_function(log_probs, torch.tensor([style_to_idx[item2]], dtype=torch.long))

#             # Step 5. Do the backward pass and update the gradient
#             loss.backward()
#             optimizer.step()

#             # Get the Python number from a 1-element Tensor by calling tensor.item()
#             total_loss += loss.item()
#         losses.append(total_loss / BATCH_SIZE)
#         print(total_loss / BATCH_SIZE)
# print(losses)

class StyleEmbedModeler(torch.nn.Module):
    
    def __init__(self, style_size, embedding_dim):
        super(StyleEmbedModeler, self).__init__()
        self.embeddings = torch.nn.Embedding(style_size, embedding_dim)
        self.linear1 = torch.nn.Linear(embedding_dim, 128)
        self.linear2 = torch.nn.Linear(128, style_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = torch.nn.functional.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = torch.nn.functional.log_softmax(out, dim=1)
        return log_probs

losses = []
loss_function = torch.nn.NLLLoss()
model = StyleEmbedModeler(len(styles), EMBEDDING_DIM)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5)
#BATCH_SIZE = 10000
#data_loader = DataLoader(pair_list, batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(10):
    print('-----------------------' + str(epoch) + '---------------------')
    total_loss = 0
    i = 0
    for item1, item2 in pair_list:
        item1_idxs = torch.tensor([style_to_idx[item1]], dtype=torch.long)
        model.zero_grad()
        log_probs = model(item1_idxs)
        loss = loss_function(log_probs, torch.tensor([style_to_idx[item2]], dtype=torch.long))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()        
        i += 1
        if not i % 5000:
            print(i)
    losses.append(total_loss)
    print(total_loss/len(pair_list))
print(losses)

-----------------------0---------------------
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000


In [ ]:
3:48
               